---
# 라이브러리 불러오기
----

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from datetime import date, datetime, timedelta
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm

- data_history = 액티브 휴폐업이력 Sheet + 휴폐업 휴폐업이력 Sheet
  
- data = 액티브 기업개요_외감 Sheet + 휴폐업 기업개요 Sheet 

- finance = 재무 데이터 수정 본

---
# 데이터셋 불러오기
----

In [ ]:
data_history = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/휴폐업이력_합본.csv')
data = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/엑티브_휴폐업_합본.csv')
finance = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/재무데이터_수정.csv')
del data_history['Unnamed: 0']
del data['Unnamed: 0']
del finance['Unnamed: 0']

In [ ]:
print('액티브 + 휴폐업 데이터셋 크기:',data.shape)
print('휴폐업 이력이 있는 데이터셋 크기:',data_history.shape)
print('재무 데이터셋 크기:',finance.shape)

액티브 + 휴폐업 데이터셋 크기: (35721, 31)
휴폐업 이력이 있는 데이터셋 크기: (11667, 9)
재무 데이터셋 크기: (109142, 53)


---
# 0. dataset 구성
---
- 예시는 아래와 같이 구성

|사업자번호||섹터||년도||기업매출||~~~||~~~||~~~||~~~|
|---||---||---||---||---||---||---||---|
|1234||A||20210||10000||10000||10000||10000||10000|
|1234||A||2021||20000||10000||10000||10000||10000|
|5678||B||2018||5000||10000||10000||10000||10000|
|5678||B||2020||17000||10000||10000||10000||10000|

---
### 0-1. 결산년월의 월이 연말(=1231)로 설정되지 않은 값 제거
----

In [ ]:
finance['결산년월'] = finance['결산년월'].astype(str)

In [ ]:
print('데이터의 수 :',len(finance))
print('결산년월이 월말(=1231)인 값의 수 :',len(finance[finance['결산년월'].str.contains('1231')]))
print('결산년월에서 제거되는 데이터 수 :',len(finance) - len(finance[finance['결산년월'].str.contains('1231')]))

데이터의 수 : 109142
결산년월이 월말(=1231)인 값의 수 : 105141
결산년월에서 제거되는 데이터 수 : 4001


In [ ]:
finance = finance[finance['결산년월'].str.contains('1231')]
finance = finance.reset_index(drop = True)
len(finance)


105141

---
### 표준산업분류(=섹터) 분류 표
----

GCIS 산업분류체계표 기준 11개를 기준으로 표준산업분류(총 21개)를 분류

1. 에너지 [Energy] = 전기,가스, 증기 및 공기 조절 공급업 (D)  

2. 소재 [Materials] = 수도,하수 및 폐기물 처리, 원료 재생업 (E)
3. 산업재 [Industrials] = 광업 (B) / 제조업 (C) / 건설업 (F) / 운수 및 창고업 (H)
4. 자유 소비재 [Discretionary] = 협회 및 단체, 수리 및 기타 개인 서비스업 (S) / 예술, 스포츠 및 여가관련 서비스업 (R) / 가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동 (T) / 교육 서비스업 (P) / 도매 및 소매업 (G)
5. 필수 소비재 [Staples] = 숙박 및 음식점업 (I) / 농업,임업 및 어업 (A)
6. 건강관리 [Health] = 보건업 및 사회복지 서비스업 (Q)
7. 금융 [Financials] = 금융 및 보험업 (K)
8. 정보기술 [Technology] = 정보통신업 (J) / 전문, 과학 및 기술 서비스업 (M)
9. 유틸리티 [Utilities] = 공공 행정, 국방 및 사회보장 행정 (O), 국제 및 외국기관 (U)
10. 부동산 [Estate] = 사업시설 관리, 사업 지원 및 임대 서비스업 (N) / 부동산업 (L)
11. 커뮤니케이션 서비스 [Communication] = 
---

In [ ]:
##### 약 11분 소요 #####

dataset = finance[['사업자번호','결산년월','휴폐업이력']]

dataset['표준산업분류'] = None

for i in tqdm(range(len(dataset))):
  if dataset['사업자번호'][i] in data['사업자등록번호'].tolist() :
    dataset['표준산업분류'][i] = data[(data['사업자등록번호'] == dataset['사업자번호'][i])]['표준산업분류'].item()

  0%|          | 0/105141 [00:00<?, ?it/s]

In [ ]:
finance['결산년월'].value_counts()

20181231    29700
20191231    26360
20211231    24583
20201231    24498
Name: 결산년월, dtype: int64

---
### 0-2. 결산년월의 표기를 년도로만 표시
----

In [ ]:
for i in range(len(dataset)):
  if dataset['결산년월'][i] == '20181231':
    dataset['결산년월'][i] = '2018'
  elif dataset['결산년월'][i] == '20191231':
    dataset['결산년월'][i] = '2019'
  elif dataset['결산년월'][i] == '20201231':
    dataset['결산년월'][i] = '2020'
  else:
    dataset['결산년월'][i] = '2021'

dataset['결산년월'] = dataset['결산년월'].astype(int)

---
# ~~재무데이터를 활용한 DCF 기업가치~~
---

    세후영업이익(NOPLAT) -> EBIT(영업이익) - TAX(법인세 비용)
    - 운전자본비용(NWC) -> 매출채권 + 재고자산 - 매입채무
    + 비현금비용(감가상각비)
    - 자본적 지출(CAPEX) -> 비유동자산
    ------------------------------------------------------------
    FCFF

----
# ~~특허 출원 여부 및 출원 수~~
----

---
# 3. 섹터별 주가 현황 데이터
---
- Selenium을 통해 네이버 금융에서 기업별 WICS 분류 크롤링

In [ ]:
kospi = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/kospi.csv')
del kospi['Sector_x'], kospi['Unnamed: 0']
kospi = kospi.dropna()
###########################################################################
kospi['code'] = kospi['code'].astype(int)
kospi['code'] = kospi['code'].astype(str)
kospi['code'] = kospi['code'].apply(lambda x : x.zfill(6))
###########################################################################
kospi = kospi[['Close','Change','date','code']]
kospi = kospi.reset_index(drop = True)

In [ ]:
kosdaq = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/kosdaq.csv')
del kosdaq['Sector_x'], kosdaq['Unnamed: 0']
kosdaq = kosdaq.dropna()
###########################################################################
kosdaq['code'] = kosdaq['code'].astype(int)
kosdaq['code'] = kosdaq['code'].astype(str)
kosdaq['code'] = kosdaq['code'].apply(lambda x : x.zfill(6))
###########################################################################
kosdaq = kosdaq[['Close','Change','date','code']]
kosdaq = kosdaq.reset_index(drop = True)

---
### 3-1. WICS를 기준으로 주식 섹터 분류
---

In [ ]:
sector = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/selenium_working.csv')
del sector['Unnamed: 0']
sector['code'] = sector['code'].astype(int)
sector['code'] = sector['code'].astype(str)
sector['code'] = sector['code'].apply(lambda x : x.zfill(6))

In [ ]:
import re 

wics = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/WICS.csv')
wics['소분류'] = wics['소분류'].apply(lambda x: re.sub(r' ', "",x))
wics['WICS'] = wics['WICS'].astype(str)

In [ ]:
count = 0
what = []
for i in range(len(sector)):
  if sector['sector'][i] not in wics['소분류'].tolist():
    count += 1
    what.append(i)

print('미분류된 개수:',count)
print('sector의 총 개수:',len(sector))
# 미분류된 값 삭제
sector = sector.drop(index = what, axis = 0)
sector = sector.reset_index(drop = True)
print('삭제 후 sector의 총 개수:',len(sector))

미분류된 개수: 5
sector의 총 개수: 2423
삭제 후 sector의 총 개수: 2418


In [ ]:
# 표준산업분류를 위한 GCIS 딕셔너리 생성
WICS_code = {'Energy' : '10','Materials' : '15','Industrials' : '20',
        'Discretionary' : '25','Staples' : '30',
        'Health' : '35','Financials' : '40','Technology' : '45',
        'Utilities' : '55','Communication' : '50'}

- WICS의 대분류를 통해 기업별 sector 분류

In [ ]:
sector['value'] = None

for i in range(len(sector)):
  index = wics[wics['소분류'] == sector['sector'][i]].index
  first = wics['WICS'][index].item()[:2]
  for key, value in WICS_code.items():
    if first in value:
      sector['value'][i] = key

- 작업시간 단축을 위해 SQL로 작업진행

      select ks."Close" as "close" , ks."Change" as "change" , split_part(ks.date::varchar, '-', 1) as date, ss.value as sector, ks.code 
      from kospi_sql ks 
      join sector_sql ss on ss.code = ks.code 

In [ ]:
raw_kospi = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/kospi_RAW.csv')
del raw_kospi['code']

kospi_sum = raw_kospi.groupby(['sector','date']).sum()
kospi_sum = kospi_sum.reset_index()

kospi_mean = raw_kospi.groupby(['sector','date']).mean()
kospi_mean = kospi_mean.reset_index()

- WICS 기준 "부동산"은 "금융"항목에 포함되기 때문에 기존 표준산업분류에서 "부동산"으로 분류된 항목들을 "금융"항목으로 변경

In [ ]:
dataset = dataset.replace('Estate','Financials')

In [ ]:
dataset['kospi_sum_close'] = None
dataset['kospi_mean_close'] = None
dataset['kospi_mean_change'] = None

for i in tqdm(range(len(dataset))):
  try:
    index_sum = kospi_sum[(kospi_sum['date'] == dataset['결산년월'][i]) & (kospi_sum['sector'] == dataset['표준산업분류'][i])].index
    index_mean = kospi_mean[(kospi_mean['date'] == dataset['결산년월'][i]) & (kospi_mean['sector'] == dataset['표준산업분류'][i])].index
    dataset['kospi_sum_close'][i] = kospi_sum['close'][index_sum].item()
    dataset['kospi_mean_close'][i] = kospi_mean['close'][index_mean].item()
    dataset['kospi_mean_change'][i] = kospi_mean['change'][index_mean].item()
  except:
    pass

  0%|          | 0/105141 [00:00<?, ?it/s]

In [ ]:
dataset.isnull().sum(0)

사업자번호                    0
결산년월                     0
휴폐업이력                98306
표준산업분류                 181
kospi_sum_close        181
kospi_mean_close       181
kospi_mean_change      181
dtype: int64

---
# 4. 기업시장지수
---
- Selenium을 통해 네이버 금융에서 기업별 WICS 분류 크롤링

In [ ]:
raw_kosdaq = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/kosdaq_RAW.csv')
del raw_kosdaq['code']
kosdaq_sum = raw_kosdaq.groupby(['sector','date']).sum()
kosdaq_sum = kosdaq_sum.reset_index()

kosdaq_mean = raw_kosdaq.groupby(['sector','date']).mean()
kosdaq_mean = kosdaq_mean.reset_index()

In [ ]:
dataset['kosdaq_sum_close'] = None
dataset['kosdaq_mean_close'] = None
dataset['kosdaq_mean_change'] = None

for i in tqdm(range(len(dataset))):
  try:
    index_sum = kosdaq_sum[(kosdaq_sum['date'] == dataset['결산년월'][i]) & (kosdaq_sum['sector'] == dataset['표준산업분류'][i])].index
    index_mean = kosdaq_mean[(kosdaq_mean['date'] == dataset['결산년월'][i]) & (kosdaq_mean['sector'] == dataset['표준산업분류'][i])].index
    dataset['kosdaq_sum_close'][i] = kosdaq_sum['close'][index_sum].item()
    dataset['kosdaq_mean_close'][i] = kosdaq_mean['close'][index_mean].item()
    dataset['kosdaq_mean_change'][i] = kosdaq_mean['change'][index_mean].item()
  except:
    pass

  0%|          | 0/105141 [00:00<?, ?it/s]

---
# 5. 기업할당지수
----

- 매출액과 자산총계 추가

In [ ]:
# 기존 재무데이터의 매출액 합치기
dataset = pd.concat([dataset,finance[['매출액','자산총계','영업이익（손실）','당기순이익(손실)']]], axis = 1)

# 데이터 타입 변환
dataset[['매출액','자산총계','영업이익（손실）','당기순이익(손실)']] = dataset[['매출액','자산총계','영업이익（손실）','당기순이익(손실)']].fillna(0).astype(int).replace({0:None})

- 년도별 섹터분류 후 총 매출액의 합 계산

In [ ]:
dataset.groupby(['결산년월','표준산업분류'])['매출액'].sum()[2018].to_dict().keys()

dict_keys(['Discretionary', 'Energy', 'Financials', 'Health', 'Industrials', 'Materials', 'Staples', 'Technology', 'Utilities'])

In [ ]:
##### 약 47분 소요 #####

dataset['기업할당지수'] = None

sector = dataset.groupby(['결산년월','표준산업분류'])['매출액'].sum()[2018].to_dict().keys()

for i in tqdm(range(len(dataset))):
  for j in sector:
    try:
      if (dataset['결산년월'][i] == 2018) & (dataset['표준산업분류'][i] == j) :
        values = dataset.groupby(['결산년월','표준산업분류'])['매출액'].sum()[2018].to_dict()
        dataset['기업할당지수'][i] = (dataset['매출액'][i] / values[j]) * 100

      elif (dataset['결산년월'][i] == 2019) & (dataset['표준산업분류'][i] == j) :
        values = dataset.groupby(['결산년월','표준산업분류'])['매출액'].sum()[2019].to_dict()
        dataset['기업할당지수'][i] = (dataset['매출액'][i] / values[j]) * 100

      elif (dataset['결산년월'][i] == 2020) & (dataset['표준산업분류'][i] == j) :
        values = dataset.groupby(['결산년월','표준산업분류'])['매출액'].sum()[2020].to_dict()
        dataset['기업할당지수'][i] = (dataset['매출액'][i] / values[j]) * 100

      elif (dataset['결산년월'][i] == 2021) & (dataset['표준산업분류'][i] == j) :
        values = dataset.groupby(['결산년월','표준산업분류'])['매출액'].sum()[2021].to_dict()
        dataset['기업할당지수'][i] = (dataset['매출액'][i] / values[j])* 100
    except:
      pass 

  0%|          | 0/105141 [00:00<?, ?it/s]

In [ ]:
dataset[(dataset['표준산업분류'] == 'Estate') & (dataset['결산년월'] == '2019')]['기업할당지수'].sum()

0

---
# 6. 기업 성장률 지표
---

---
### 6-1 연평균 성장률 (CAGR)
---

- (최신연도 매출액/설립연도 매출액)^(1/(최신연도-설립연도))-1

In [ ]:
# 기존 재무데이터에서 만들어 놓은 매출액 성장률 합치기
dataset = pd.concat([dataset,finance['매출액성장률']], axis = 1)
dataset.isnull().sum()

사업자번호                     0
결산년월                      0
휴폐업이력                 98306
표준산업분류                  181
kospi_sum_close         181
kospi_mean_close        181
kospi_mean_change       181
kosdaq_sum_close        181
kosdaq_mean_close       181
kosdaq_mean_change      181
매출액                    5222
자산총계                    808
영업이익（손실）                910
당기순이익(손실)               866
기업할당지수                 5394
매출액성장률                38444
dtype: int64

---
### 6-2 기업 안정성 평가
---

- **매출 성장률** = (당해년도 매출액 / 직전년도 매출액) x 100 - 100
- **총자산 성장률** = (당해년도 총자산 / 직전년도 총자산) x 100 - 100

In [ ]:
############ 사업자번호가 중복되는 값의 인덱스를 변환하기 위한 코드 ############

# 사업자번호의 중복값의 시작값은 True
a = dataset['사업자번호'].duplicated()

# 사업자번호 전체 인덱스 값 생성
b = list(range(len(a)))

for i in range(len(a)):
  if a[i] == True: # True 라는 것은 사업자번호가 중복되는 값의 시작점이며 다음 False가 나올기 전 까지 중복된 값임
    b.remove(i)

print('재무제표 전체 데이터 수:',len(dataset))
print('중복을 제외한 사업자번호 수:',len(b))

재무제표 전체 데이터 수: 105141
중복을 제외한 사업자번호 수: 34751


In [ ]:
##### 약 1분 소요 #####

dataset['매출 성장률'] = None

for i in range(len(b)):
  num = b[i]
  if num == 105137 :
    num2 = 105141
  else: 
    num2 = b[i+1]
  count = num2 - num

  for j in range(count):
    try:
      if j+1 != count:
        dataset['매출 성장률'][num + j + 1] = round(((dataset['매출액'][num + j + 1] - dataset['매출액'][num + j]) / dataset['매출액'][num + j]) * 100 ,3)
      else :
        pass
    except:
      pass

In [ ]:
##### 약 1분 소요 #####

dataset['총자산 성장률'] = None

for i in range(len(b)):
  num = b[i]
  if num == 105137 :
    num2 = 105141
  else: 
    num2 = b[i+1]
  count = num2 - num

  for j in range(count):
    try:
      if j+1 != count:
        dataset['총자산 성장률'][num + j + 1] = round(((dataset['자산총계'][num + j + 1] - dataset['자산총계'][num + j]) / dataset['자산총계'][num + j]) * 100 ,3)
      else :
        pass
    except:
      pass

---
# 7. 기업의 존속기간별 구간
---

- 9 = 1년 이하 
- 8 = 1년 ~ 2년 
- 7 = 2년 ~ 3년 
- 6 = 3년 ~ 4년 
- 5 = 4년 ~ 5년 
- 4 = 5년 ~ 7년 
- 3 = 7년 ~ 10년 
- 2 = 10년 ~ 15년 
- 1 = 15년 ~ 20년 
- 0 = 20년 이상 



In [ ]:
##### 약 9분 소요 #####

dataset['존속기간'] = None

for i in tqdm(range(len(dataset))):
  if dataset['사업자번호'][i] in data['사업자등록번호'].tolist() :
    dataset['존속기간'][i] = data[(data['사업자등록번호'] == dataset['사업자번호'][i])]['존속기간'].item()

# 데이터 타입 변환
dataset['존속기간'] = dataset['존속기간'].fillna(0).astype(int).replace({0:None})

  0%|          | 0/105141 [00:00<?, ?it/s]

In [ ]:
##### 약 1분 소요 #####

for i in range(len(dataset)):
  try:
    if dataset['결산년월'][i] == "2020":
      dataset['존속기간'][i] = dataset['존속기간'][i] - 300
    elif dataset['결산년월'][i] == "2019":
      dataset['존속기간'][i] = dataset['존속기간'][i] - 665
    elif dataset['결산년월'][i] == "2018":
      dataset['존속기간'][i] = dataset['존속기간'][i] - 1030
  except:
    pass 

In [ ]:
index = dataset[dataset['존속기간'] <= 365].index
dataset['존속기간'][index] = 9
    
index = dataset[(dataset['존속기간'] > 365) & (dataset['존속기간'] <= 730)].index
dataset['존속기간'][index] = 8

index = dataset[(dataset['존속기간'] > 730) & (dataset['존속기간'] <= 1095)].index
dataset['존속기간'][index] = 7

index = dataset[(dataset['존속기간'] > 1095) & (dataset['존속기간'] <= 1460)].index
dataset['존속기간'][index] = 6

index = dataset[(dataset['존속기간'] > 1460) & (dataset['존속기간'] <= 1825)].index
dataset['존속기간'][index] = 5

index = dataset[(dataset['존속기간'] > 1825) & (dataset['존속기간'] <= 2555)].index
dataset['존속기간'][index] = 4

index = dataset[(dataset['존속기간'] > 2555) & (dataset['존속기간'] <= 3650)].index
dataset['존속기간'][index] = 3

index = dataset[(dataset['존속기간'] > 3650) & (dataset['존속기간'] <= 5475)].index
dataset['존속기간'][index] = 2

index = dataset[(dataset['존속기간'] > 5475) & (dataset['존속기간'] <= 7300)].index
dataset['존속기간'][index] = 1

index = dataset[(dataset['존속기간'] > 7300)].index
dataset['존속기간'][index] = 0

In [ ]:
dataset.isnull().sum()

사업자번호                     0
결산년월                      0
휴폐업이력                 98306
표준산업분류                  181
kospi_sum_close         181
kospi_mean_close        181
kospi_mean_change       181
kosdaq_sum_close        181
kosdaq_mean_close       181
kosdaq_mean_change      181
매출액                    5222
자산총계                    808
영업이익（손실）                910
당기순이익(손실)               866
기업할당지수                 5394
매출액성장률                38444
매출 성장률                38712
총자산 성장률               35317
존속기간                   1855
dtype: int64

---
# 8. ~~Target값 생성~~
----

- 휴폐업 발생 년도 이후 값만 1로 레이블링 할때 사용한 코드

In [ ]:
dataset = dataset.replace('close',1)
dataset = dataset.replace('active',1)

In [ ]:
for i in tqdm(range(len(dataset))):
  if pd.isna(dataset['휴폐업이력'][i]):
    dataset['휴폐업이력'][i] = 0

dataset['휴폐업이력'] = dataset['휴폐업이력'].astype(int)
del dataset['표준산업분류']

  0%|          | 0/105141 [00:00<?, ?it/s]

In [ ]:
import pandas as pd 
final = pd.read_csv('/content/drive/MyDrive/Numble_Challenge/data/dataset_final.csv')
del final['Unnamed: 0']
final = final.dropna()
final.reset_index(drop = True)
final

,사업자번호,결산년월,휴폐업이력,kospi_sum_close,kospi_mean_close,kospi_mean_change,kosdaq_sum_close,kosdaq_mean_close,kosdaq_mean_change,매출액,자산총계,영업이익（손실）,당기순이익(손실),기업할당지수,매출액성장률,매출 성장률,총자산 성장률,존속기간
1,1018100340,2019,0,2.873155e+08,20446.593937,-0.000105,2.605542e+07,3993.780196,0.000445,6541583.0,36320470.0,843072.0,2733421.0,0.012519,-1.33,-1.326,3.731,0.0
2,1018100340,2020,0,2.470454e+08,16303.400977,0.000400,2.994878e+07,3617.876299,0.001440,5933317.0,40601940.0,68146.0,3406757.0,0.009986,-5.40,-9.298,11.788,0.0
3,1018100340,2021,0,3.343307e+08,19855.725680,0.001174,5.422458e+07,4547.897761,0.001125,6158586.0,38310842.0,387000.0,3823473.0,0.008187,-2.43,3.797,-5.643,0.0
5,1018100772,2019,0,1.003511e+09,71566.878191,0.000020,1.505344e+08,14398.318317,0.000432,1150528.0,21376102.0,239972.0,11534.0,0.017053,2.08,2.079,2.808,0.0
6,1018100772,2020,0,8.743153e+08,61706.208272,0.000761,1.385269e+08,12184.615973,0.001187,662177.0,21548434.0,-665914.0,-717899.0,0.018388,-23.35,-42.446,0.806,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105131,8998800150,2020,0,6.018734e+08,36986.011983,0.001615,1.250373e+09,10145.261118,0.001819,24980062.0,29223901.0,-810985.0,-247235.0,0.069473,-9.93,-49.970,-19.279,2.0
105132,8998800150,2021,0,9.182064e+08,54596.644072,0.000916,1.923732e+09,14587.649317,0.000844,25680342.0,27021362.0,930018.0,922054.0,0.059379,-5.87,2.803,-7.537,2.0
105134,8998800427,2019,0,9.142384e+08,22062.270325,0.000068,3.274347e+08,7686.080491,0.000350,15869364.0,13176486.0,995588.0,579237.0,0.004317,66.02,66.018,26.062,4.0
105135,8998800427,2020,0,8.304181e+08,19474.181582,0.001118,3.213642e+08,7231.906857,0.001332,18863062.0,15145761.0,1272377.0,535114.0,0.005525,40.48,18.865,14.945,4.0


In [ ]:
final.to_csv('/content/drive/MyDrive/Numble_Challenge/data/dataset_정제_후.csv')